In [129]:
import pandas as pd
import numpy as np
import urllib.request
import requests
from bs4 import BeautifulSoup
import re
from scipy.spatial.distance import cosine
from numpy import dot
from numpy.linalg import norm


URL = "https://computersecurityclass.com/"

In [20]:
#Task 1.1
#creating the DataFrame from the .dat file
original_csv = pd.read_table("user_artists.dat", sep="\t", usecols=['userID', 'artistID', 'weight'])

original_csv.dropna()
original_csv.drop_duplicates()

print(original_csv)
original_csv.to_csv("csv_user_artists.csv")

       userID  artistID  weight
0           2        51   13883
1           2        52   11690
2           2        53   11351
3           2        54   10300
4           2        55    8983
...       ...       ...     ...
92829    2100     18726     337
92830    2100     18727     297
92831    2100     18728     281
92832    2100     18729     280
92833    2100     18730     263

[92834 rows x 3 columns]


In [22]:
#Task 1.2
#creating function to scrape the data for a single user
#scraper helper function
def scraping(user):
    temp_url = URL + user
    #print(temp_url)
    temp_text = requests.get(temp_url).text
    soup = BeautifulSoup(temp_text, 'html.parser')
    #print(soup.prettify())

    return_arr = []

    for link in soup.find_all('li'):
        #print(link.get('href'))
        #print(link.get_text())
        temp_arr = []
        splitline = link.get_text().split('has been listened to')
        #print(splitline)

        artist_name = splitline[0][0:len(splitline[0]) - 1]
        #print(artist_name)
        temp_arr.append(artist_name)

        times_listened = int(re.sub(",","",splitline[1][1:len(splitline[1]) - 7]))
        #print(times_listened)
        temp_arr.append(times_listened)

        return_arr.append(temp_arr)

    return return_arr

In [23]:
#opening URL
page = requests.get(URL)
text = page.text

#opening using beautiful soup
soup = BeautifulSoup(text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   Fan Pages!
  </title>
  <meta charset="utf-8"/>
  <style type="text/css">
   body {
 background: black;
 font-family: sans-serif;
 color: white;
 margin: 30px;
}
 h1 {
 font-size: 45pt;
}
h3 {
 font-size: 15pt;
}
a:link{
 color: #EEEEEE;
 font-size: 15pt;
} 
a:visited{
 color: #CCCCCC;
 font-size: 15pt;
}
  </style>
 </head>
 <body>
  <h3>
   Welcome to the best music fan page on the internet! Here, you can see listening stats for the following members:
   <br/>
   <br/>
   <li>
    <a href="Aaren.html">
     Aaren
    </a>
   </li>
   <li>
    <a href="Abbey.html">
     Abbey
    </a>
   </li>
   <li>
    <a href="Abbi.html">
     Abbi
    </a>
   </li>
   <li>
    <a href="Abbie.html">
     Abbie
    </a>
   </li>
   <li>
    <a href="Abby.html">
     Abby
    </a>
   </li>
   <li>
    <a href="Abigale.html">
     Abigale
    </a>
   </li>
   <li>
    <a href="Abra.html">
     Abra
    </a>
   </li>
   <li>
    <a href="Ada.html">
     Ada


In [24]:
#creating a list of href objects to check within the names
users = []
for link in soup.find_all('a'):
    #print(link.get('href'))
    users.append(link.get('href'))

print(users)

['Aaren.html', 'Abbey.html', 'Abbi.html', 'Abbie.html', 'Abby.html', 'Abigale.html', 'Abra.html', 'Ada.html', 'Adaline.html', 'Adel.html', 'Adelaide.html', 'Adeline.html', 'Adella.html', 'Adelle.html', 'Adena.html', 'Adey.html', 'Adora.html', 'Adoree.html', 'Adrea.html', 'Adriaens.html', 'Adrianna.html', 'Adrianne.html', 'Adriena.html', 'Adrienne.html', 'Agata.html', 'Aggy.html', 'Agnella.html', 'Agnes.html', 'Agretha.html', 'Aigneis.html', 'Aile.html', 'Ailene.html', 'Ailey.html', 'Ailina.html', 'Ainsley.html', 'Ainslie.html', 'Ajay.html', 'Alaine.html', 'Alana.html', 'Alane.html', 'Alayne.html', 'Alberta.html', 'Albertina.html', 'Albertine.html', 'Alejandra.html', 'Alene.html', 'Alessandra.html', 'Alethea.html', 'Alexa.html', 'Alexandra.html', 'Alexandrina.html', 'Alexina.html', 'Alexine.html', 'Alfy.html', 'Alia.html', 'Alidia.html', 'Alika.html', 'Alli.html', 'Allie.html', 'Allina.html', 'Allissa.html', 'Allx.html', 'Almeria.html', 'Almeta.html', 'Almira.html', 'Almire.html', 'Alve

In [25]:
csv_scraped =  pd.DataFrame(columns = ['user','artist','weight'])
for user in users:
    #print(user[0:len(user) - 5])
    #print(scraping(user))
    for scrapes in scraping(user):
        add = pd.DataFrame({
                    'user': [user[0:len(user) - 5]],
                    'artist' : [scrapes[0]],
                    'weight' : [scrapes[1]]})
        csv_scraped = pd.concat([csv_scraped, add], ignore_index = True, axis = 0)
    
print(csv_scraped)
csv_scraped.to_csv("scraped.csv")

         user                                             artist weight
0       Aaren                                             Ab Ovo   1160
1       Aaren                                           Aes Dana    338
2       Aaren                                    All India Radio    617
3       Aaren                                          Alva Noto    346
4       Aaren  Alva Noto + Ryuichi Sakamoto with Ensemble Modern    216
...       ...                                                ...    ...
69965  ZsaZsa                                              Tarja   1700
69966  ZsaZsa                                       Toni Braxton    786
69967  ZsaZsa                                  Within Temptation   1948
69968  ZsaZsa                                       audiomachine   1357
69969  ZsaZsa                                           t.A.T.u.    887

[69970 rows x 3 columns]


In [125]:
#creating dictionary for the original given data
#working with original_csv and csv_scraped

#looping through original_csv
og_userid = original_csv['userID'].tolist()
og_artistid = original_csv['artistID'].tolist()
og_weight = original_csv['weight'].tolist()

curr_userid = og_userid[0]

i = 0
og_dict = dict()

while i < len(og_userid):
    #looping through each user
    curr_user_id = og_userid[i]
    curr_artist_id  = og_artistid[i]
    curr_weight = og_weight[i]

    #print(curr_user_id)
    #print(curr_artist_id)
    #print(curr_weight)
    
    #print(type(curr_user_id))
    #print(type(curr_artist_id))
    #print(type(curr_weight))
   
    #check if you already have a dictionary for that user
    #if og_dict[curr_user_id] is not None:
    if curr_user_id in og_dict:
        temp_dict = og_dict[curr_user_id]
        temp_dict[curr_artist_id] = curr_weight
        og_dict[curr_user_id] = temp_dict
    else:
        temp_dict = dict()
        temp_dict[curr_artist_id] = curr_weight
        og_dict[curr_user_id] = temp_dict
    
    i = i + 1
        
    """"
    temp_dict = dict()
    temp_dict[og_artistid[i]] = og_weight[i]
    if og_userid[i] == curr_userid:
        temp_dict[og_artistid[i]] = og_weight[i]
        curr_userid = og_userid[i]
        #print(temp_dict)
    else:
        #print(curr_userid)
        #print(temp_dict)
        #print(og_userid[i])
        #print()
        og_dict[curr_userid] = temp_dict
        #add_dict = {curr_userid: temp_dict}
        #print(add_dict)
        #og_dict.update(add_dict)
        temp_dict.clear()
        curr_userid = og_userid[i]
        #print(curr_userid)
        #print()
    i = i + 1
    """
    
print(og_dict)
#print(temp_dict)

{2: {51: 13883, 52: 11690, 53: 11351, 54: 10300, 55: 8983, 56: 6152, 57: 5955, 58: 4616, 59: 4337, 60: 4147, 61: 3923, 62: 3782, 63: 3735, 64: 3644, 65: 3579, 66: 3312, 67: 3301, 68: 2927, 69: 2720, 70: 2686, 71: 2654, 72: 2619, 73: 2584, 74: 2547, 75: 2397, 76: 2382, 77: 2120, 78: 2119, 79: 1990, 80: 1972, 81: 1948, 82: 1868, 83: 1792, 84: 1740, 85: 1638, 86: 1594, 87: 1559, 88: 1553, 89: 1519, 90: 1471, 91: 1438, 92: 1411, 93: 1407, 94: 1373, 95: 1363, 96: 1342, 97: 1337, 98: 1332, 99: 1330, 100: 1315}, 3: {101: 13176, 102: 662, 103: 493, 104: 431, 105: 403, 106: 354, 107: 269, 108: 236, 109: 215, 110: 215, 111: 212, 112: 193, 113: 187, 114: 167, 115: 163, 116: 154, 117: 151, 118: 142, 119: 142, 120: 125, 121: 111, 122: 108, 123: 104, 124: 99, 125: 98, 126: 94, 127: 89, 128: 89, 129: 86, 130: 85, 131: 84, 132: 83, 133: 83, 134: 77, 135: 77, 136: 76, 137: 75, 138: 72, 139: 72, 140: 71, 141: 70, 142: 70, 143: 70, 144: 69, 145: 68, 146: 67, 147: 67, 148: 66, 149: 66, 150: 65}, 4: {51: 2

In [106]:
#creating dictionary for the scraped, new data
#working with original_csv and csv_scraped


#looping through csv_scraped
scraped_user = csv_scraped['user'].tolist()
scraped_artist = csv_scraped['artist'].tolist()
scraped_weight = csv_scraped['weight'].tolist()

#print(scraped_user)
#print(scraped_artist)
#print(scraped_weight)

#print(len(scraped_user))
#print(len(scraped_artist))
#print(len(scraped_weight))

#mapping each artist name to an artist id
index = 0
artistid_mapping = dict()
for artist in scraped_artist:
    if not artist in artistid_mapping:
        artistid_mapping[artist] = index
        index = index + 1
print(artistid_mapping)

{'Ab Ovo': 0, 'Aes Dana': 1, 'All India Radio': 2, 'Alva Noto': 3, 'Alva Noto + Ryuichi Sakamoto with Ensemble Modern': 4, 'Asura': 5, 'Between Interval': 6, 'Biosphere': 7, 'Carbon Based Lifeforms': 8, "Dustin O'Halloran": 9, 'Flying Lotus': 10, 'H.U.V.A. Network': 11, 'Helios': 12, 'Hol Baumann': 13, 'Jagjit Singh': 14, 'Komet': 15, 'Library Tapes': 16, 'Lusine': 17, 'Lusine ICL': 18, 'Massive Attack': 19, 'Melba Moore': 20, 'Nega': 21, 'Nene Altro & O Mal de Caim': 22, 'Nino Katamadze': 23, 'Nino Katamadze & Insight': 24, 'Orsten': 25, 'Oval': 26, 'Phaeleh': 27, 'Quidam': 28, 'R.D.Burman': 29, 'Radiohead': 30, 'RelespÃºblica': 31, 'Ryoji Ikeda': 32, 'Si': 33, 'Sigur RÃ³s': 34, 'Silencide': 35, 'Spiraal Aurel': 36, 'The Boats': 37, 'The Idan Raichel Project': 38, 'The Tiger Lillies': 39, 'Ulrich Schnauss': 40, 'Zoviet France': 41, 'ksandr': 42, 'Aeternom': 43, 'AlemÃ£o Ronaldo': 44, 'Alexisonfire': 45, 'Armandinho': 46, 'Armin van Buuren': 47, 'Avril Lavigne': 48, 'BidÃª ou Balde': 4

In [126]:
#creating the dictionary for the scraped data

#scraped_user = csv_scraped['user'].tolist()
#scraped_artist = csv_scraped['artist'].tolist()
#scraped_weight = csv_scraped['weight'].tolist()

curr_user = scraped_user[0]

i = 0
scraped_dict = dict()
temp_dict = dict()
while i < len(scraped_user):
    #looping through each user
    curr_user_id = scraped_user[i]
    curr_artist_id  = scraped_artist[i]
    curr_weight = scraped_weight[i]
    
    #check if you already have a dictionary for that user
    #if og_dict[curr_user_id] is not None:
    if curr_user_id in scraped_dict:
        temp_dict = scraped_dict[curr_user_id]
        temp_dict[artistid_mapping.get(curr_artist_id)] = curr_weight
        scraped_dict[curr_user_id] = temp_dict
    else:
        temp_dict = dict()
        temp_dict[artistid_mapping.get(curr_artist_id)] = curr_weight
        scraped_dict[curr_user_id] = temp_dict
    
    i = i + 1
    
print(scraped_dict)

{'Aaren': {0: 1160, 1: 338, 2: 617, 3: 346, 4: 216, 5: 163, 6: 155, 7: 488, 8: 342, 9: 229, 10: 181, 11: 488, 12: 827, 13: 475, 14: 364, 15: 174, 16: 392, 17: 174, 18: 1412, 19: 165, 20: 373, 21: 1172, 22: 954, 23: 227, 24: 400, 25: 152, 26: 426, 27: 742, 28: 413, 29: 168, 30: 220, 31: 1030, 32: 206, 33: 154, 34: 324, 35: 569, 36: 275, 37: 209, 38: 835, 39: 401, 40: 565, 41: 722, 42: 253}, 'Abbey': {43: 58165, 44: 693, 45: 470, 46: 322, 47: 280, 48: 1673, 49: 1248, 50: 2702, 51: 106760, 52: 565, 53: 438, 54: 35204, 55: 4520, 56: 1516, 57: 483, 58: 611, 59: 3644, 60: 1924, 61: 50554, 62: 63223, 63: 334, 64: 71409, 65: 1241, 66: 79550, 67: 557, 68: 276, 69: 1649, 70: 1044, 71: 519, 72: 2518, 73: 92079, 74: 18106, 75: 1284, 76: 33596, 77: 859, 78: 697, 79: 319, 80: 3984, 81: 636, 82: 26029, 83: 386, 84: 967, 85: 41719, 86: 2405, 87: 60985}, 'Abbi': {88: 1518, 89: 1061, 90: 714, 91: 1884, 92: 121, 93: 1011, 94: 1386, 95: 2514, 96: 2229, 97: 2502, 98: 594, 99: 981, 100: 939, 101: 218, 102: 

In [181]:
#calculating cosine similarity
import pandas as pd
from scipy.spatial.distance import cosine
from scipy import spatial
import math
import numpy as np
import json
#cosine similarity dataframe
matched = pd.DataFrame(columns = ['id','name'])

#nested for loops to loop through the two dictionaries to compare values
#og_dict and scraped_dict
def norma(dct):
    return math.sqrt(sum(x*x for x in dct.values()))
        
def cosine_similarity(dict_1, dict_2):
    intersecting_keys = list(dict_1.keys() & dict_2.keys())

    List1 = list(dict_1[k] for k in intersecting_keys)
    List2 = list(dict_2[k] for k in intersecting_keys)
    
    similarity = np.dot(List1,List2) / (norma(dict_1) * norma(dict_2))
    return round(similarity, 2)

number_of_matches = 0

for og_key, og_value in og_dict.items():
    for scraped_key, scraped_value in scraped_dict.items():
        #print(og_value)
        #print(scraped_value)
        #result = cosine_similarity(og_value, scraped_value)
        #result = 1 - spatial.distance.cosine(og_value, scraped_value)
        #print(type(og_value))
        #print(type(scraped_value))
        result = cosine_similarity(og_value, scraped_value)
        #print(result)
        if result >= 0.5:
            #print(result)
            #print(og_key)
            #print(scraped_key)
            data = {'id': [og_key], 'name': [scraped_key]}
            pd.Series(data).to_frame()
            temp_matched = pd.DataFrame(data, columns = ['id', 'name'])
            #print(temp_matched)
            #print(type(temp_matched))
            #print(type(matched))
            
            matched = pd.concat([temp_matched, matched], axis=0)
            
            number_of_matches = number_of_matches + 1

#print(number_of_matches)
print(matched)


      id      name
0   2001  Violetta
0   2001    Carita
0   1866     Tilly
0   1866     Linda
0   1866      Hedy
..   ...       ...
0     16    Adoree
0     14      Adey
0     10   Adeline
0      9      Adel
0      2     Abbey

[70 rows x 2 columns]


In [192]:
#writing the matched dictionary to a json file
#json = json.dumps(matched)

data = matched.to_json(orient='records')
#print(data)
#print(type(data))

#json_object = json.dumps(matched, indent = 4)
  
# Writing to sample.json
with open("matched.json", "w") as outfile:
    outfile.write(data)

#json = matched.to_json(orient="records")
#parsed = json.loads(json)
#with open("matched.json", "w") as outfile:
    #json.dump(parsed, outfile)